In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'image-matching-challenge-2024:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F71885%2F8069805%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240328%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240328T180900Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D5c8d10397c7ab2110803fbcfebffd453301fb163b7e2fed2d2f5711adcf381a86770f676b68f18aca1d44a5183241418b0a68c9c2d8fb8a6dcdbcc10a157ed2f05b92e4170ae7e26ed87e91693251b1b9f79f65fc63f3c6f2e6ae77b46f923ca07361492e6ffad52a0273f97e868d83d657689adff86eb9758803b62dedd77e0e73b3ca44e69c83946fc3fde18c10b6b27358f399afffa56dea58732a7b1c9750c8969b297f3619205fa7c3f0e6ca0cbc9b05d8bbd21fc51ef8b4d3257a4338e8dd2fc60855efa6166c5aa2158d96a4e966c6e65957b7c480b4f5973faa5503904820fb340a3f756b2ba17ef900c06e3d9ebea92a5db0d19e28c808168c03d1a'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


# 📸 Image Matching Challenge - 📊 Exploratory Data Analysis

Hoş geldiniz! Bu heyecan verici yarışmada, 2D görüntülerden 3D sahneleri oluşturma yeteneğinizi test edeceğiz. Ancak önce bazı önemli bilgilere göz atalım:

# 🚂 Eğitim ve Test Veri Kümesi İncelemesi

Hoş geldiniz! Bu veri kümesi, eşsiz mekanlarda çekilmiş bir dizi görüntü içerir. Bazı eğitim veri setleri, "images_full" adlı bir klasörde ek görüntüler içerebilir. Yayınlanan test klasörü ise eğitimdeki kilise sahnesinin bir alt kümesini içerir ve yalnızca örnek amaçlar için sağlanmıştır. Eğitim verisi genellikle ardışık bir sırayla çekilen ve önemli ölçüde içerik örtüşmesine sahip görüntülerden oluşurken, test seti sınırlı görüntü örtüşmesine ve rastgele bir sıraya sahiptir.

## 📁 Eğitim Veri Kümesi Klasörleri:
- **images:** Aynı konumun yakınında çekilmiş bir dizi görüntü.
- **smf:** Bu görüntü grubu için bir 3D rekonstrüksiyon, bu yarışma ile birlikte paketlenmiş olan 3D hareket yapısından-colmap adlı kütüphane ile açılabilir.
- **LICENSE.txt:** Bu veri kümesinin lisansı.

## 📄 Eğitim Etiketleri CSV Dosyası (`train_labels.csv`):
### 1️⃣ Eğitim Etiketleri İncelemesi:
- **dataset:** Veri kümesi için benzersiz bir tanımlayıcı.
- **scene:** Sahne için benzersiz bir tanımlayıcı.
- **image_path:** Dosya adı ve yolu dahil görüntü dosya adı.
- **rotation_matrix:** İlk hedef sütunu. Satır majörü kurallarına göre düzleştirilmiş, noktalı virgül ile ayrılmış değerler içeren bir 3x3 matris vektörü.
- **translation_vector:** İkinci hedef sütunu. Noktalı virgül ile ayrılmış değerlere sahip 3 boyutlu bir vektör.

## **Hedefler:**
1. **Hassas 3D Haritalar Oluşturma:** Farklı senaryo ve ortamlardan gelen görüntü setlerinden doğru mekansal temsiller oluşturmak için bir model geliştirmek.
2. **Yapıdan Hareketle (SfM):** Çeşitli görüntülerin bir koleksiyonundan bir ortamın 3D modelini yeniden oluşturma süreci.
3. **Çeşitli Görüntü Kaynaklarını Keşfetme:** Dronlar 🤖, yoğun ormanlar 🌲 ve gece vakti 🌙 gibi gerçekçi ve uygulanabilir senaryolardan görüntülerle çalışma.

Organizatörler, bu yarışma için farklı zorluklar içeren 6 kategori belirlediler:
1. 🏛️ **Foto Turizmi ve Tarihi Koruma:** Farklı bakış açıları, sensör tipleri, günün/zamanın saati ve örtüler. Antik tarihi siteler eşsiz zorluklar ekler.
2. ☀️ **Gece ve Gündüz ve Zamansal Değişimler:** Gündüz ve gece fotoğraflarının birleşimi, kötü aydınlatma veya aylar/yıllar arasında farklı hava koşullarında çekilmiş fotoğraflar.
3. ✈️ **Hava ve Karışık Hava-Yer:** Dronlardan gelen görüntüler, rastgele düzlem dışı dönüşler, benzer görüntülerle eşleştirilmiş ve ayrıca yerden çekilmiş görüntüler.
4. ♻️ **Tekrarlanan Yapılar:** Simetrik nesneler, perspektifi ayırt etmek için ayrıntılara ihtiyaç duyar.
5. 🌲 **Doğal Ortamlar:** Ağaçlar ve bitki örtüsü gibi düzensiz yapılar.
6. 🪞 **Şeffaflıklar ve Yansımalar:** Cam eşyalar gibi doku eksikliği ve farklı bir dizi sorun oluşturan yansımalar ve yansımalar.

Hazır mısınız? Görüntüleri eşleştirmek için hazırlanın ve 3D dünyayı yeniden keşfedin! 🚀


# Install & Import dependencies 📦🔍

In [ ]:
!pip install -q mediapy

In [ ]:
%cd /kaggle/working/
!rm -rf /kaggle/working/Hierarchical-Localization
!git clone --quiet --recursive https://github.com/cvg/Hierarchical-Localization/
%cd /kaggle/working/Hierarchical-Localization
!pip install -e .

from hloc import extract_features, match_features, reconstruction, visualization, pairs_from_exhaustive
from hloc.visualization import plot_images, read_image
from hloc.utils import viz_3d

%cd /kaggle/working/

In [ ]:
from pathlib import Path

import cv2
import mediapy
import pandas as pd
import plotly.express as px
import pycolmap

# 📊 Advanced Dataset Exploration with Emojis 📈

In [ ]:
# Path to the train_labels.csv file
train_labels_path = "/kaggle/input/image-matching-challenge-2024/train/train_labels.csv"

In [ ]:
# Load the CSV file into a DataFrame
train_labels_df = pd.read_csv(train_labels_path)

In [ ]:
# Display the first few rows of the DataFrame
print("First few rows of train_labels.csv:")
print(train_labels_df.head())

# 2️⃣ Exploring the Interplay Between Datasets and Scenes 🔄📊

In [ ]:
# Calculate the number of unique scenes within each dataset
scenes_per_dataset = train_labels_df.groupby('dataset')['scene'].nunique()

In [ ]:
train_labels_df.groupby("dataset")["scene"].nunique()

In [ ]:
# Import necessary library for plotting
import matplotlib.pyplot as plt

# Define a custom color palette
colors = plt.cm.tab10.colors

# Visualize the relationship between datasets and scenes
plt.figure(figsize=(12, 8))
scenes_per_dataset.sort_values(ascending=False).plot(kind='bar', color=colors)
plt.title('Number of Unique Scenes per Dataset', fontsize=16, fontweight='bold')
plt.xlabel('Dataset', fontsize=14)
plt.ylabel('Number of Unique Scenes', fontsize=14)
plt.xticks(rotation=45, ha='right', fontsize=12)
plt.yticks(fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# 🔍📊 Delving into Train Dataset Categories 🌟

In [ ]:
train_categories_df = pd.read_csv("/kaggle/input/image-matching-challenge-2024/train/categories.csv")
train_categories_df

In [ ]:
# Check for missing or corrupted data
train_categories_df.isnull().sum()

In [ ]:
# Handle missing data appropriately
train_categories_df.dropna(inplace=True)  # We can directly drop the missing values

In [ ]:
# Check the first few observations in the dataset
print(train_categories_df.head())

In [ ]:
# Split the categories column by semicolon and explode it into separate rows
train_categories_df['category'] = train_categories_df['categories'].str.split(';')
train_categories_df = train_categories_df.explode('category')

# Drop the original categories column since it's no longer needed
train_categories_df.drop(columns=['categories'], inplace=True)

# Display the updated DataFrame
train_categories_df.head()

In [ ]:
import seaborn as sns

# Determine the number of categories
num_categories = len(train_categories_df['category'].unique())

# Visualize the relationship between scenes and categories
plt.figure(figsize=(14, 10))
sns.countplot(y='category', hue='scene', data=train_categories_df, palette='viridis')
plt.title('Relationship Between Scenes and Categories', fontsize=16)
plt.xlabel('Count', fontsize=14)
plt.ylabel('Category', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.legend(title='Scene', fontsize=12, title_fontsize=12)
plt.show()

# 📊📈 Visualizing Train Dataset Categories 🎨


In [ ]:
# Count the occurrences of each category
category_counts = train_categories_df['category'].value_counts()

# Create a bar plot
plt.figure(figsize=(12, 8))
sns.barplot(x=category_counts.index, y=category_counts.values, palette='viridis')
plt.title('Distribution of Categories in the Train Dataset', fontsize=16)
plt.xlabel('Category', fontsize=14)
plt.ylabel('Count', fontsize=14)
plt.xticks(rotation=45, ha='right', fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.font_manager as fm

# Create a pivot table to count the occurrences of each category in each scene
scene_category_counts = train_categories_df.pivot_table(index='scene', columns='category', aggfunc='size', fill_value=0)

# Define a custom color palette
custom_palette = sns.light_palette("seagreen", as_cmap=True)

# Set custom font styles
title_font = {'fontname': 'Times New Roman', 'fontsize': 16, 'fontweight': 'bold'}
label_font = {'fontname': 'Arial', 'fontsize': 14}

# Set the figure style
sns.set_style("white")

# Plot the heatmap with improved aesthetics
plt.figure(figsize=(14, 10))
sns.heatmap(scene_category_counts, cmap=custom_palette, cbar=True, linewidths=0.5, linecolor='gray', annot=True, fmt='d', annot_kws={'fontsize': 10})
plt.title('Relationship Between Scenes and Categories', **title_font)
plt.xlabel('Category', **label_font)
plt.ylabel('Scene', **label_font)
plt.xticks(rotation=45, ha='right', **label_font)
plt.yticks(**label_font)
plt.gca().patch.set_facecolor('lightgray')  # Set background color
plt.tight_layout()
plt.show()

## Dive into each dataset! 🚀

In [ ]:
def explore(split: str, dataset: str, plot_image_limit: int = 12) -> None:
    # Define the path to the dataset
    path = Path("/kaggle/input/image-matching-challenge-2024") / split / dataset
    images_path = path / "images"
    smf_path = path / "smf"

    # Load and display images
    images = [cv2.cvtColor(cv2.imread(str(p)), cv2.COLOR_BGR2RGB) for p in list(images_path.glob("*"))[:plot_image_limit]]
    mediapy.show_images(images, height=300, columns=3)

    # If not the test split, visualize 3D reconstruction
    if split != "test":
        rec_gt = pycolmap.Reconstruction(smf_path)
        fig = viz_3d.init_figure()
        viz_3d.plot_reconstruction(fig, rec_gt, cameras=False, color='rgba(227,168,30,0.5)', name="Ground Truth", cs=5)
        fig.show()

In [ ]:
explore(split="train", dataset="church")

In [ ]:
explore(split="test", dataset="church")

In [ ]:
explore(split="train", dataset="dioscuri")

In [ ]:
explore(split="train", dataset="lizard")

In [ ]:
explore(split="test", dataset="lizard")

In [ ]:
explore(split="train", dataset="pond")

In [ ]:
explore(split="train", dataset="transp_obj_glass_cup")

In [ ]:
explore(split="train", dataset="transp_obj_glass_cylinder")